In [1]:
from tensorflow.keras.models import load_model
import numpy as np
from skimage import transform, io, img_as_float, exposure, morphology, img_as_ubyte
import matplotlib.pyplot as plt
import os
from PIL import Image
from shutil import move
import cv2

# 新段落

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Load model
model_name = '/content/drive/MyDrive/MEDICAL/trained_model.hdf5'
UNet = load_model(model_name,compile=False)

In [4]:
def loadDataGeneral(path, im_shape):
    """Function for loading arbitrary data in standard formats"""
    X = []
    #image = Image.open(path).convert('RGB')
    image = cv2.imread(path)
    image = np.array(image)
    img = img_as_float(image)
    img = transform.resize(img, im_shape)
    img = exposure.equalize_hist(img)
    img = np.expand_dims(img, -1)
    X.append(img)
    X = np.array(X)
    X -= X.mean()
    X /= X.std()
    return X, img

In [5]:
def remove_small_regions(img, size):
    """Morphologically removes small (less than size) connected regions of 0s or 1s."""
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
im_shape = (256, 256)
path = '/content/drive/MyDrive/data/train'
path2 = '/content/drive/MyDrive/data/removesegmented'
path3 = '/content/drive/MyDrive/data/brokenfiles'
output_dir = '/content/drive/MyDrive/data/orgimagessegmented'
arr = os.listdir(os.path.join(path))
for j,file in enumerate(arr):
    try:
        image_name = os.path.join(path, file)
        X,img_org = loadDataGeneral(image_name, im_shape)
        img = exposure.rescale_intensity(np.squeeze(X), out_range=(0,1))
        img = np.sum(img,axis=2)[None,:,:,None]
        pred = UNet.predict(img)
        #-------------------------
        pr = pred > 0.5
        # Remove regions smaller than 2% of the image
        pr = remove_small_regions(pr, 0.02 * np.prod(im_shape))
        pr2 = np.squeeze(pr)
        seg_img = np.squeeze(img_org)*pr2[:,:,None]#np.dstack((pr2, pr2, pr2))
        maximo = np.max(seg_img)
        minimo = np.min(seg_img)
        if (maximo==0) and (minimo == 0):
            #print(file)
            move(image_name,os.path.join(path2, file))
        else:
            seg_img /= maximo
            #seg_img = (seg_img - minimo)/(maximo-minimo)
            #io.imshow(seg_img)
            save_path = os.path.join(output_dir, file)
            io.imsave(save_path,img_as_ubyte(seg_img))
    except:
        move(image_name, os.path.join(path3, file))
        #break

/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:438: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel, or set channel_axis.
  return func(*args, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━

KeyboardInterrupt: 